In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


food_url = 'https://tarkov-market.com/tag/food'
drink_url = 'https://tarkov-market.com/tag/drinks'
# URL to be scraped
food_page = requests.get(food_url)
drink_page = requests.get(drink_url)


food_soup = BeautifulSoup(food_page.content, 'html.parser')
drink_soup = BeautifulSoup(drink_page.content, 'html.parser')


In [2]:
food_table_list_div = food_soup.find('div', class_='table-list')
drink_table_list_div = drink_soup.find('div', class_='table-list')

In [3]:
provisions_price_dict = {}

In [ ]:
all_food_rows = food_table_list_div.find_all('div', class_='row')
all_drink_rows = drink_table_list_div.find_all('div', class_='row')

    # Iterate over each child div
for child_div in all_food_rows[1:]:
    # Do something with each child div
    #print(child_div.prettify())
    price = child_div.find('span', class_='price-main')
    price_num = re.sub(r'\D', '', price.text)
    print(price_num)
    print(price.text)
    name = child_div.find('span', class_='name').text.replace('"', '')
    print(name)
    print('-------------------')
    if name == 'Bottle of Tarkovskaya vodka (bad)':
        continue
    provisions_price_dict[name] = int(price_num)

for child_div in all_drink_rows[1:]:
    # Do something with each child div
    #print(child_div.prettify())
    price = child_div.find('span', class_='price-main')
    price_num = re.sub(r'\D', '', price.text)
    print(price_num)
    print(price.text)
    name = child_div.find('span', class_='name').text.replace('"', '')
    print(name)
    print('-------------------')
    if name == 'Bottle of Tarkovskaya vodka (bad)':
        continue
    provisions_price_dict[name] = int(price_num)
    
    

In [ ]:
from items_dict import items_dict

for item in items_dict.items():
    print(item)

In [ ]:
for item in provisions_price_dict.items():
    print(item)

In [ ]:
full_dict = {}
for key, price_24h_avg in provisions_price_dict.items():
    energy, hydration = items_dict[key]
    full_dict[key] = [price_24h_avg, energy, hydration]
    print(f"name: {key}, price: {price_24h_avg}, energy: {energy}, hydration: {hydration}")


In [9]:
# Set column names
columns = ['name', 'price_24h_avg', 'hydration', 'energy']

df = pd.DataFrame.from_dict(full_dict, orient='index', columns=['price', 'energy', 'hydration']).reset_index()

# Rename the 'index' column to 'name'
df = df.rename(columns={'index': 'name'})


================================================================================================================================
DF ready
================================================================================================================================

In [14]:
def best_to_consume(current_energy, current_hydration, min_energy, min_hydration, df):
    """
    Returns the best item to consume based on the current energy and hydration levels.
    """
    # Calculate the difference between current energy and hydration levels and 


    dp_arr = [[float('inf') for i in range(total_hydration_columns)] for j in range(total_energy_rows)]

current_energy, current_hydration, min_energy, min_hydration = 60, 60, 90, 90

energy_diff = min_energy - current_energy
hydration_diff = min_hydration - current_hydration
total_energy_rows = energy_diff + 100
total_hydration_columns = hydration_diff + 100

dp_arr = [[float('inf') for i in range(total_hydration_columns)] for j in range(total_energy_rows)]
dp_arr[0][0] = 0

valid_start_points = [(0,0)]

while valid_start_points:
    start_energy_row, start_hydration_column = valid_start_points.pop(0)
    print(f"trying starting point (e,h): {start_energy_row}, {start_hydration_column}")
    for name, price, energy, hydration in df.values:

        if start_energy_row >= energy_diff and start_hydration_column >= hydration_diff:
            print(f"  starting point too far, removing (e,h): {start_energy_row}, {start_hydration_column}")
            continue

        print(f"  starting point valid, jumping from (e,h): {start_energy_row}, {start_hydration_column}")
        result_energy_row = start_energy_row + energy
        result_hydration_column = start_hydration_column + hydration

        #check for out of bounds/negative effects
        if result_energy_row >= total_energy_rows or result_hydration_column >= total_hydration_columns or result_energy_row < 0 or result_hydration_column < 0:
            print(f"  provision lands out of bounds, skipping (e,h): {result_energy_row}, {result_hydration_column}. name: {name}, price: {price}, energy: {energy}, hydration: {hydration}")
            continue
        
        curr_combo_price = dp_arr[start_energy_row][start_hydration_column] + price

        if dp_arr[result_energy_row][result_hydration_column] > curr_combo_price:
            dp_arr[result_energy_row][result_hydration_column] = curr_combo_price

            valid_start_points.append((result_energy_row, result_hydration_column))

            for energy_row in range(0, result_energy_row):
                for hydration_column in range(0, result_hydration_column):
                    if energy_row < start_energy_row and hydration_column < start_hydration_column:
                        continue
                    if dp_arr[energy_row][hydration_column] > curr_combo_price:
                        dp_arr[energy_row][hydration_column] = curr_combo_price
print(f"price at target: {dp_arr[energy_diff][hydration_diff]}")

            
        




trying starting point (e,h): 0, 0
  starting point valid, jumping from (e,h): 0, 0
  starting point valid, jumping from (e,h): 0, 0
  provision lands out of bounds, skipping (e,h): 40, -13. name: Pack of oat flakes, price: 23025, energy: 40, hydration: -13
  starting point valid, jumping from (e,h): 0, 0
  provision lands out of bounds, skipping (e,h): 50, -5. name: Can of humpback salmon, price: 14441, energy: 50, hydration: -5
  starting point valid, jumping from (e,h): 0, 0
  provision lands out of bounds, skipping (e,h): 10, -10. name: Emelya rye croutons, price: 8948, energy: 10, hydration: -10
  starting point valid, jumping from (e,h): 0, 0
  provision lands out of bounds, skipping (e,h): 50, -5. name: Salty Dog beef sausage, price: 17995, energy: 50, hydration: -5
  starting point valid, jumping from (e,h): 0, 0
  provision lands out of bounds, skipping (e,h): 48, -2. name: Can of pacific saury, price: 12203, energy: 48, hydration: -2
  starting point valid, jumping from (e,h):

In [ ]:
for name, price, energy, hydration in df.values:
    print(name, price, energy, hydration)

In [ ]:
def best_to_consume(current_energy, current_hydration, goal_energy, goal_hydration, df):
    max_energy = 100  # Maximum energy value
    max_hydration = 100  # Maximum hydration value

    # Initialize a 2D array to store minimum prices
    dp = [[[float('inf'), []] for _ in range(max_hydration + 1)] for _ in range(max_energy + 1)]
    dp[current_energy][current_hydration] = [0, []]

    # Iterate over all provisions
    for name, price, energy, hydration in df.values:
        price = int(price)
        energy = int(energy)
        hydration = int(hydration)
        name = str(name)
        for i in range(max_energy, -1, -1):
            for j in range(max_hydration, -1, -1):
                new_i = min(i + energy, max_energy)
                new_j = min(j + hydration, max_hydration)
                total_price, chosen_provisions = dp[i][j]
                new_total_price = total_price + price

                if new_total_price < dp[new_i][new_j][0]:
                    dp[new_i][new_j] = [new_total_price, chosen_provisions + [name]]

    # Find the minimum total price to reach the goal
    min_price, chosen_provisions = dp[goal_energy][goal_hydration]

    return min_price, chosen_provisions

# Example usage
current_energy = 34
current_hydration = 25
goal_energy = 91
goal_hydration = 96

min_price, chosen_provisions = best_to_consume(current_energy, current_hydration, goal_energy, goal_hydration, df)

print(f"The minimum total price is: {min_price}")
print("Chosen provisions:")
for provision in chosen_provisions:
    print(provision, '      RUB',df[df['name'] == provision]['price'].values[0])